In [289]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as optimize
import scipy.special as special
import matplotlib.pyplot as plt


In [23]:
os.getcwd()
#os.chdir('w09_EM')

'/Users/mpresley84/Desktop/w09_EM'

In [6]:
# 1. Generate Simulator

# 2. Calculate the log likehood if the model and the parameters were known
#    a. explain the steps of your calculation
#    b. use your LL for the simulated data for your own parameter values
#    c. compare Lestrade data to your actual data for your simualtor
#    d. compare LL of Lestrade to your own of the true postitive control (yours should be better)

# 3. Estimate isoform abundances by EM
#    a. estimates the unkown isoform abundance 𝝉 for each isoform (Ar1..Arc10). given read couts r and the sturucture of the Arc locus including the lengths Li, using expectation maximation
#.   b. apply your Emto lestrade supplementary data
#.   c. what are the most abundant transcripts
#.   d. what are the least abundant transcripts
#.   e. Explain why lestrade's data is wrong,


In [35]:
#Mpresley 5/10/23
# 1. Simulator

#a.import model, as Panda table
#b. Create a distribution of Direction that will create 10 values that sum up to one for the simualtor
#theta = np.random.dirichlet(np.ones(10))
Seq = ["A","B","C","D","E","F","G","H","I","J"]
Arc_locs = {'Arc1':'ABCD','Arc2':'BC','Arc3':'CDE','Arc4':'DEFG','Arc5':'EFGH','Arc6':'EGH','Acr7':'GH','Acr8':'HI','Acr9':'IJA','Arc10':'JAB'}
Arc_keys = list(Arc_locs.keys())
#taus = {Arc_keys[i]: theta[i] for i in range(len(Arc_locs.keys()))}
os.

{'Arc1': 0.01180967282153745,
 'Arc2': 0.02732962316129933,
 'Arc3': 0.027995994879347962,
 'Arc4': 0.2838608278561838,
 'Arc5': 0.10066596297998968,
 'Arc6': 0.367245947991555,
 'Acr7': 0.013413781539448288,
 'Acr8': 0.13627917267556278,
 'Acr9': 0.024431646613959697,
 'Arc10': 0.00696736948111594}

In [83]:

#list("ABC")
np.random.choice(list(Arc_locs.values()))


#np.random.choice(["ABC".split()])
#    res.append(np.random.choice(["A","B","C","D","E"]))


'JAB'

In [211]:
def GenerateRead(N_abunds,isoForms):
    loc = np.random.choice(isoForms,1,p=N_abunds)
    read = np.random.choice(list(loc[0]))
    return read
    

In [ ]:
GenerateRead()

In [213]:
res = []
for i in range(100000):
    res.append(GenerateRead(list(taus.values()) ,list(Arc_locs.values())))

In [223]:
def GenerateModel(Taus,Arcs,N):
    #get Lengths from Arcs
    res = []
    seq = ['A','B','C','D','E','F','G','H','I','J']
    res_dict = {}
    Arc_lens = [len(arc) for arc in Arc_locs.values()]
    Ns = [Taus[i] * Arc_lens[i] for i in range(len(Taus))]
    N_abunds = Ns/sum(Ns)
    print(N_abunds)
    print(Arc_locs.values())
    for i in range(N):
        res.append(GenerateRead(N_abunds,list(Arc_locs.values())))
    for s in seq:
        result_dict[s] = res.count(s)
    return result_dict
    
    

    

In [530]:
#Mpresley generated on 5/15/23 
simulated_Reads = GenerateModel(list(taus.values()),Arc_locs,1000000)


#list(taus.values())
#Arc_locs


[0.01467353 0.01697854 0.02608878 0.3526973  0.12507754 0.34222753
 0.00833332 0.08466349 0.02276725 0.00649272]
dict_values(['ABCD', 'BC', 'CDE', 'DEFG', 'EFGH', 'EGH', 'GH', 'HI', 'IJA', 'JAB'])


In [535]:
#MPresley 5/15/23
print(simulated_Reads)
print(taus)


{'A': 13256, 'B': 14300, 'C': 21079, 'D': 100157, 'E': 243221, 'F': 119442, 'G': 237388, 'H': 192134, 'I': 49468, 'J': 9555}
{'Arc1': 0.01180967282153745, 'Arc2': 0.02732962316129933, 'Arc3': 0.027995994879347962, 'Arc4': 0.2838608278561838, 'Arc5': 0.10066596297998968, 'Arc6': 0.367245947991555, 'Acr7': 0.013413781539448288, 'Acr8': 0.13627917267556278, 'Acr9': 0.024431646613959697, 'Arc10': 0.00696736948111594}


In [170]:
[len(arc) for arc in Arc_locs.values() ]
#len(list(Arc_locs.values())[0]

[4, 2, 3, 4, 4, 3, 2, 2, 3, 3]

In [230]:
Seq
result_dict = {}
for s in Seq:
    print(res.count(s))
    result_dict[s] = res.count(s)
print(result_dict)
    


1310
1902
2654
8241
22549
9730
22644
22276
7632
1062
{'A': 1310, 'B': 1902, 'C': 2654, 'D': 8241, 'E': 22549, 'F': 9730, 'G': 22644, 'H': 22276, 'I': 7632, 'J': 1062}


In [293]:
#Mpresley 5/12/23, let's try to program the loglikehood script using the student's homework (needed a little bit of help here)
def getSegmentProbDict (theta,Segments,Arcs):
    #theta = nucleotide abundances, so for example [.3,.2,.3.,.1,.1] (should add up to one, but there is 10 of then and not 5 like in this example
    #Segment will be a...j , up to 10
    #Arcs is the Arc structure, will be some combination of the segments, so for exampe {ab,bc,bcd,cd} etc should also add up to 10.
    
    #first let's get the lengths of the Arcs
    arc_lengths_dict = {}
    theta_dict = {}
    SegmentProbDict  = {}
    #Make dictionaries for easy referencing
    for a in Arcs:
        arc_lengths_dict[a] = len(a)
    
    for i in range(len(theta)):
        theta_dict[Arcs[i]] = theta[i]
    
    
    #iterative through the lengths
    for s in Segments:
        s_list = []
        for a in Arcs:
            if s in a:
                s_list.append(np.log(theta_dict[a] * 1/arc_lengths_dict[a]))

        SegmentProbDict[s] = s_list
    return SegmentProbDict

In [261]:
Segments = ['A','B','C','D','E','F','G','H','I','J']
Arcs = list(Arc_locs.values())


In [294]:

segtest = getSegmentProbDict(theta,Segments,Arcs)


In [312]:
def getSumProbArray(SegmentProbDict):
    S_likehood = {}
    for s in SegmentProbDict:
        S_likehood[s] = special.logsumexp(SegmentProbDict[s])
    return S_likehood
#special.logsumexp(segtest)

In [313]:
#getSumProbArray(segtest)

{'A': -4.311101781852516,
 'B': -3.966495755261106,
 'C': -3.651613416280882,
 'D': -2.4859116731042294,
 'E': -1.4789404399974544,
 'F': -2.3420361765064897,
 'G': -1.4905270486774285,
 'H': -1.5031505491087094,
 'I': -2.573299028092897,
 'J': -4.559591009778708}

In [344]:
def getTotalProbArray(S_likehood,read_counts):
    S_likehood_total = []
    for s in S_likehood:
        S_likehood_total.append(S_likehood[s] * read_counts[s])
    return sum(S_likehood_total)

In [317]:
def getLL(theta,Arcs,read_counts):
    Segments = list(read_counts.keys())
    segmentProbDict =  getSegmentProbDict(theta,Segments,Arcs)
    S_likehood = getSumProbArray(segmentProbDict)
    
    return getTotalProbArray(S_likehood,read_counts)
    

In [320]:
read_counts = {'A':97512,'B':268771,'C':249157,'D':76643,'E':33977,'F':40729,'G':43351,'H':76299,'I':73029,'J':40532,}

In [346]:
#getSegmentProbDict(theta,segments,arcs)
getTotalProbArray(S_likehood,read_counts)

-3284494.2300386424

In [352]:
testResults = getLL(theta,Arcs,read_counts)

In [355]:
testResults

-3284494.2300386424

In [364]:
theta[Segments.index("C")]
#Segments.index("C")


0.027995994879347962

In [765]:
#3. Let's generate the EM function
def Expectation(theta,read_counts,Arcs):
    #read_counts should be a dictionar of counts
    #theta should be an array that sums up to 1 
    #Arcs should be an array of the Arc Transcripts ex ["ABCD","BC"...etc]
    
    #first let's create the segmetns from the keys of the read_counts dictionary
    Segments = list(read_counts.keys())
    Arc_lengths = [len(a) for a in Arcs]
      

    #let's creata theta dictionary from the theta array and the Segments list
    theta_dict = {}
    for s in Segments:
        theta_dict[s] = theta[Segments.index(s)]
    prob_dict = {}
    prob_singleRead_array = []
    joint_prob = getSegmentProbDict(theta,Segments,Arcs)
    sProbs = getSumProbArray(joint_prob)
    for s in Segments:
        prob_singleRead_array = []
        for t in Arcs:
            if s in t:
                num = np.log(theta_dict[s] * (1/Arc_lengths[Arcs.index(t)]))
                denum = sProbs[s]
                prob_singleRead_array.append(num-denum)
            else:
                prob_singleRead_array.append(0)
       # prob_singleRead_array = np.array(prob_singleRead_array)       
#        prob_singleRead_array = prob_singleRead_array/prob_singleRead_array.sum()
        prob_dict[s] = np.array(prob_singleRead_array)
    #Now we have the probablity dictionary matrix of one read. posterior probablity matrix, lets multiple the reads
    prob_dict_sums = {}
    for s in prob_dict:
        prob_dict_sums[s] = prob_dict[s] * read_counts[s]
#print(prob_dict[s] * read_counts[s])
    return prob_dict_sums
        
        

In [855]:
testExpect = Expectation2(theta,read_counts,Arcs)
#testarray = [5,3,5]
#testarray2 = np.array([5,3,5])
#print(testarray2*5)
#testExpect['A']
#arraytest = np.array([25,5,3,15,40])
#arraytest/arraytest.sum()

In [861]:
print(testExpect['A'])
print(log_step1['A'])

[-1.51402893  0.          0.          0.          0.          0.
  0.          0.         -1.22634686 -1.22634686]
[-1.5140289319347904, 0, 0, 0, 0, 0, 0, 0, -0.49938650149123287, -1.7540280381265347]


In [456]:
#np.array(testarray) 
testexpect_pd = pd.DataFrame(testExpect)
#testexpect_pd.count()
len(testexpect_pd.index)
#for i in range(10):
 #   print(i)
#sum(testexpect_pd.iloc[0])


10

In [483]:
#Mpresley 5/12/23, attached is the maxmize step
def Maxmization(postprobs):
    #postprobs is a dictionary
    # The 1st step is to convert the postprobs dictionary into a panda dataframe
    postprobs_df = pd.DataFrame(postprobs)
    theta = []
    # The 2nd step is to sum the columns for the thetas
    for i in range(len(postprobs_df.index)):
        theta.append(sum(postprobs_df.iloc[i]))
    # 3rd we will normalize the theta array
    normalized_theta = [t/sum(theta) for t in theta] 
    return normalized_theta

In [1081]:
def EM(theta,read_counts,Arcs,converge=None,maxrounds=100):
    #rounds 
    old_ll = 0
    for r in range(maxrounds):
        counts = Expectation2(theta,read_counts,Arcs)
        theta = Maxmization2(counts)
        ll = getLL(theta,Arcs,read_counts)
        delta = (ll - old_ll)
        old_ll = ll
    #    print('delta',delta)
        if converge != None and abs(delta) < converge:
      #      print('converge, delta', converge,delta)
       #     print('ll, theta',ll,theta)
            return(ll,theta)
    return(ll,theta)
        

In [499]:
def testFunc(a,b=None):
    if b == None:
        return(a)
    else:
        return(a+b)

In [608]:
#testFunc("hello")
abs(-5)

5

In [496]:
def testFunc(a,b):
    return(a+b)

In [1088]:
#result_10_round = EM(theta,read_counts,Arcs,None,10)
result_100_round_b = EM(theta,read_counts,Arcs,1,100)
result_100_round_c = EM(theta,read_counts,Arcs,None,10000)
#result_1000_round = EM(theta,read_counts,Arcs,None,1000)
print('round_b',result_100_round_b[0])
print('round_c',result_100_round_c[0])

round_b -2022308.9707417663
round_c -2022239.2255290712


In [669]:
#result_100_round_b
Arcs

['ABCD', 'BC', 'CDE', 'DEFG', 'EFGH', 'EGH', 'GH', 'HI', 'IJA', 'JAB']

In [1090]:
print(result_100_round_c[0])

-2022239.2255290712


In [513]:
result_1_round

(-2423044.0239493162,
 [0.21904943050965234,
  0.04213141141136058,
  0.14188395515547392,
  0.23956671365801732,
  0.17273699913628762,
  0.06354567288622934,
  0.032301275714605185,
  0.03583963757454473,
  0.014201801469384942,
  0.038743102484443834])

In [516]:
result_10_round

(-2236414.7036050637,
 [0.32747509156516685,
  0.0842940815677223,
  0.04718782522175288,
  0.010509815055131173,
  0.022609784678229324,
  0.015002958947175873,
  0.008377551374874151,
  0.043388716287826194,
  0.20542472144719814,
  0.23572945385492325])

In [521]:
#result_100_round

In [532]:
#using Lestrade code (this is for question 2 b. use Lestrade to process the your data (what your simulator made), b.ii compare it to the actuall thetas, use the Log-Likehood
#what are the most abundant two transcripts, c. What are the least two abundant ones 4. what do you think of Lestrade conclusion

#! /usr/bin/env python3

# w09-naive.py
#   The analysis done by Lestrade et al.
#   Also serves as an example of parsing the input data.
# 
# Usage:
#   ./w09-naive.py <infile>
#
# Estimate the number of reads assigned to each isoform by a naive
# method.  Assign each read to its cognate segment, trusting the
# mapping and ignoring accuracy; then assign to isoform
# uniformly. (i.e. if 3 isoforms share the same segment, assign 1/3
# count to each isoform.)
#

import numpy as np
import string
import sys
import re

# Parse the input file.
#
with open(sys.argv[1]) as f:
    #   The first line is "The <n> transcripts of the sand mouse Arc locus"
    line  = f.readline()
    match = re.search(r'^The (\d+) transcripts', line)
    T     = int(match.group(1))

    # The next T lines are 
    #   <Arcn>  <true_tau> <L> <structure>
    # tau's may be present, or obscured ("xxxxx")
    tau       = np.zeros(T)
    L         = np.zeros(T).astype(int)
    tau_known = True   # until we see otherwise
    for i in range(T):
        fields    = f.readline().split()
        if fields[1] == "xxxxx":
            tau_known = False
        else:
            tau[i] = float(fields[1])
        L[i]      = int(fields[2])

    # after a blank line,
    # 'The <n> read sequences':
    line  = f.readline()
    line  = f.readline()
    match = re.search(r'The (\d+) read sequences', line)
    N     = int(match.group(1))

    # the next T lines are 
    #  <read a-j> <count>
    r = np.zeros(T).astype(int)
    for k in range(T):
        fields = f.readline().split()
        r[k]   = fields[1]


S = T    # S = R = T : there are T transcripts (Arc1..Arc10), S segments (A..J), R reads (a..j)
R = T
Slabel   = list(string.ascii_uppercase)[:S]               # ['A'..'J']        : the upper case labels for Arc locus segments 
Tlabel   = [ "Arc{}".format(d) for d in range(1,T+1) ]    # ['Arc1'..'Arc10'] : the labels for Arc transcript isoforms
Rlabel   = list(string.ascii_lowercase)[:T]               # ['a'..'j']        : lower case labels for reads


# Count how often each segment A..J is used in the isoforms i
# We'll use that to split observed read counts across the isoforms
# that they might have come from.
#
segusage = np.zeros(S).astype(int)
for i in range(T):
    for j in range(i,i+L[i]): 
        segusage[j%S] += 1


# Naive analysis:
#
c  = np.zeros(T)
for i in range(T):
    for k in range(i,i+L[i]):
        c[i] += (1.0 / float(segusage[k%S])) * float(r[k%S])  # For each read k, assume read k-> segment j,
                                                              # and assign 1/usage count to each transcript
                                                              # that contains segment j.
Z       = np.sum(c)
est_nu  = np.divide(c, Z)       # nucleotide abundance

print(c)

est_tau = np.divide(est_nu, L)  # convert to TPM, transcript abundance
est_tau = np.divide(est_tau, np.sum(est_tau))

# Print a table of the resulting estimates for tau
for i in range(T):
    print ("{0:10s}  {1:5.3f}".format(Tlabel[i], est_tau[i]))


FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [539]:
#Mpresley 5/15/23, out for w09-naive.py, from data of simulated_Reads 
'''
[ 49597.33333333  11793.         121485.66666667 213620.33333333
 228268.16666667 147194.5        107380.5         72767.5
  33930.16666667  13962.83333333]


Arc1        0.038
Arc2        0.018
Arc3        0.125
Arc4        0.165
Arc5        0.176
Arc6        0.151
Arc7        0.166
Arc8        0.112
Arc9        0.035
Arc10       0.014


#Mpresley 5/15/23, true arcs
{'Arc1': 0.01180967282153745,
 'Arc2': 0.02732962316129933,
 'Arc3': 0.027995994879347962,
 'Arc4': 0.2838608278561838,
 'Arc5': 0.10066596297998968,
 'Arc6': 0.367245947991555,
 'Acr7': 0.013413781539448288,
 'Acr8': 0.13627917267556278,
 'Acr9': 0.024431646613959697,
 'Arc10': 0.00696736948111594}
 
 true reads

'''
#taus
# True Counts
"'A': 13256, 'B': 14300, 'C': 21079, 'D': 100157, 'E': 243221, 'F': 119442, 'G': 237388, 'H': 192134, 'I': 49468, 'J': 9555}"
'''
{'A': 13256,
 'B': 14300,
 'C': 21079,
 'D': 100157,
 'E': 243221,
 'F': 119442,
 'G': 237388,
 'H': 192134,
 'I': 49468,
 'J': 9555} '''



"\n{'A': 13256,\n 'B': 14300,\n 'C': 21079,\n 'D': 100157,\n 'E': 243221,\n 'F': 119442,\n 'G': 237388,\n 'H': 192134,\n 'I': 49468,\n 'J': 9555} "

In [542]:
##testResults = getLL(theta,Arcs,read_counts)
#geeting getLL()

In [621]:
'''
Arc1        0.038
Arc2        0.018
Arc3        0.125
Arc4        0.165
Arc5        0.176
Arc6        0.151
Arc7        0.166
Arc8        0.112
Arc9        0.035
Arc10       0.014 '''
Lestrade_theta = {'Arc1' : .038,
                  'Arc2' : .018,
                  'Arc3' : .125,
                  'Arc4' : .165,
                  'Arc5' : .176,
                  'Arc6' : .151,
                  'Arc7' : .166,
                  'Arc8' : .112,
                  'Arc9' : .035,
                  'Arc10' : .014}

#getLL(Lestrade_theta, Arcs, simulated_Reads )
#simulated_Reads
#read_counts
#type(theta)
#Lestrade_theta_array = np.array(list(Lestrade_theta.values()))
#Lestrade_theta_array
#getLL(Lestrade_theta_array,Arcs,simulated_Reads)
# The ouput is -1928094.320718252, of the LL
#looking at the true taus
#getLL(taus,Arcs,simulated_Reads)
#taus_array = np.array(list(taus.values()))

#getLL(taus_array,Arcs,simulated_Reads) gave an output of -1892771.93797029,
# this gives a difference of 35,323



In [663]:
# result_100_round_c this was used by getting a converge of .01 delta (pretty small amount)
#The thetas for this are 
print("The thetas are ",result_100_round_c[1])
results = list(result_100_round_c[1])


The thetas are  [0.3301361442294943, 0.08673428285972554, 0.053980871079123936, 0.013160703883100566, 0.023380865096103758, 0.0153264454492793, 0.008343305500636267, 0.04160639614917142, 0.19800192447969064, 0.22932906127367433]


In [665]:
#Arcs
for a in Arcs:
    Arc_Results[a] = results[Arcs.index(a)]
Arc_Results



{'ABCD': 0.3301361442294943,
 'BC': 0.08673428285972554,
 'CDE': 0.053980871079123936,
 'DEFG': 0.013160703883100566,
 'EFGH': 0.023380865096103758,
 'EGH': 0.0153264454492793,
 'GH': 0.008343305500636267,
 'HI': 0.04160639614917142,
 'IJA': 0.19800192447969064,
 'JAB': 0.22932906127367433}

In [666]:
#The highest transcript are ABCD and JAB and account for about 50 percent of the expression levels"
# The lowest transcript are DEFG and GH,
#BC and CDE are both expressed at pretty low levels.

In [769]:
read_counts 

{'A': 97512,
 'B': 268771,
 'C': 249157,
 'D': 76643,
 'E': 33977,
 'F': 40729,
 'G': 43351,
 'H': 76299,
 'I': 73029,
 'J': 40532}

In [952]:
def logprob_T(nu, transcriptome, S):
    L = [len(t) for t in transcriptome]
    logprob_Ts = {}
    
    joint_loglikelihoods = logjoint(nu, transcriptome, S)
    S_loglikelihoods = logS(joint_loglikelihoods)
        
    for s in S:
        logprob_Ts[s] = []
        for t in range(0, len(transcriptome)):
            if s in transcriptome[t]:
                num = np.log((1 / L[t]) * nu[t])
                denom = S_loglikelihoods[s]
                logprob_Ts[s].append(num - denom)
            else:
                logprob_Ts[s].append(0)
    
    return logprob_Ts

In [844]:
def assign_counts(logprob_Ts, reads, S):
    counts = [0] * 10
    
    for s in logprob_Ts:
        read_s = reads[s]
        for t in range(0, len(logprob_Ts[s])):
            if logprob_Ts[s][t] != 0:
                counts[t] += read_s * np.exp(logprob_Ts[s][t])
                
    return counts

In [782]:
def update_nu(assigned_counts):
    total = sum(assigned_counts)
    return [c/total for c in assigned_counts]

In [904]:
# step 1 calculate log P(S, T| v, L) for each combination of S and T
def logjoint(nu, transcriptome, S):
    L = [len(t) for t in transcriptome]
    joint_likelihoods = {}
    for s in S:
        joint_likelihoods[s] = []
        
        # for each transcript t
        for t in range(0, len(transcriptome)): 
            if s in transcriptome[t]:
                # only consider if the segment is present in the transcript 
                log_likelihood = np.log((1 / L[t]) * nu[t])
                joint_likelihoods[s].append(log_likelihood)
    
    return joint_likelihoods

In [785]:
# step 2 calculate log P(S | v, L) by marginalizing over T
def logS(joint_loglikelihoods):
    S_likelihoods = {}
    for s in joint_loglikelihoods:
        S_likelihoods[s] = special.logsumexp(joint_loglikelihoods[s])
    
    return S_likelihoods

In [777]:
#logprob_T
#Expectation(theta,read_counts,Arcs)
#logprob_T(nu, transcriptome, S)
#Segments = list(read_counts.keys())

In [954]:
log_step1 = logprob_T(theta,Arcs,Segments)

denom -4.311101781852516
denom -4.311101781852516
denom -4.311101781852516
denom -3.966495755261106
denom -3.966495755261106
denom -3.966495755261106
denom -3.651613416280882
denom -3.651613416280882
denom -3.651613416280882
denom -2.4859116731042294
denom -2.4859116731042294
denom -2.4859116731042294
denom -1.4789404399974544
denom -1.4789404399974544
denom -1.4789404399974544
denom -1.4789404399974544
denom -2.3420361765064897
denom -2.3420361765064897
denom -1.4905270486774285
denom -1.4905270486774285
denom -1.4905270486774285
denom -1.4905270486774285
denom -1.5031505491087094
denom -1.5031505491087094
denom -1.5031505491087094
denom -1.5031505491087094
denom -2.573299028092897
denom -2.573299028092897
denom -4.559591009778708
denom -4.559591009778708


In [849]:
#logprob_Ts, reads, S
#log_step2 = assign_counts(log_step1,read_counts,Segments)

#log_step1.index('A')
#read_counts[Segments.index('A')]







s A
0
s B
1
s C
2
s D
3
s E
4
s F
5
s G
6
s H
7
s I
8
s J
9


In [1077]:
log_step2

[94418.58521193822,
 325121.41229417035,
 99585.98385676125,
 119638.46661803014,
 27891.055153880494,
 83803.37110737103,
 3591.414938186735,
 88606.30524181349,
 98514.79713156736,
 58828.60844628089]

In [1074]:
def Expectation2(theta,read_counts,Arcs):
    #read_counts should be a dictionar of counts
    #theta should be an array that sums up to 1 
    #Arcs should be an array of the Arc Transcripts ex ["ABCD","BC"...etc]
    
    #first let's create the segmetns from the keys of the read_counts dictionary
    Segments = list(read_counts.keys())
    Arc_lengths = [len(a) for a in Arcs]
      

    #let's creata theta dictionary from the theta array and the Segments list
    theta_dict = {}
    for a in Arcs:
        theta_dict[a] = theta[Arcs.index(a)]
    prob_dict = {}
    prob_singleRead_array = []
    joint_prob = getSegmentProbDict(theta,Segments,Arcs)
    sProbs = getSumProbArray(joint_prob)
    for s in Segments:
        prob_singleRead_array = []
        for t in Arcs:
            if s in t:
                num = np.log(theta_dict[t] * (1/Arc_lengths[Arcs.index(t)]))
                denum = sProbs[s]
                prob_singleRead_array.append(num-denum)
            else:
                prob_singleRead_array.append(0)
       # prob_singleRead_array = np.array(prob_singleRead_array)       
#        prob_singleRead_array = prob_singleRead_array/prob_singleRead_array.sum()
        prob_dict[s] = np.array(prob_singleRead_array)
    #Now we have the probablity dictionary matrix of one read. posterior probablity matrix, lets multiple the reads
    prob_dict_sums = {}
    counts = np.zeros(len(prob_dict.keys()))
    for s in prob_dict:
        for i in range(len(prob_dict[s])):
            if s in Arcs[i]:
                 counts[i] += (np.exp(prob_dict[s][i]) * read_counts[s])
    return counts
#        prob_dict_sums[s] = np.sum(np.exp(prob_dict[s]) * read_counts[s])
#print(prob_dict[s] * read_counts[s])
#    total = sum(counts)
    
#    for i in range(len(counts)):
#        counts[i] = counts[i]/total
#    return counts

In [1075]:
testExpect = Expectation2(theta,read_counts,Arcs)
#print(testExpect['B'])
#print(log_step1['B'])
#log_step1 = logprob_T(theta,Arcs,Segments)
#testarray = [5,3,5]
#np.exp(testarray)
#np.exp(3)


In [1076]:
testExpect

array([ 94418.58521194, 325121.41229417,  99585.98385676, 119638.46661803,
        27891.05515388,  83803.37110737,   3591.41493819,  88606.30524181,
        98514.79713157,  58828.60844628])

In [1034]:
log_step2

[94418.58521193822,
 325121.41229417035,
 99585.98385676125,
 119638.46661803014,
 27891.055153880494,
 83803.37110737103,
 3591.414938186735,
 88606.30524181349,
 98514.79713156736,
 58828.60844628089]

In [972]:
#segments
log_step2 = assign_counts(log_step1,read_counts,Segments)

np.exp(logprob_Ts[s][t]) 0.2200217372267056
np.exp(logprob_Ts[s][t]) 0.6069028795343667
np.exp(logprob_Ts[s][t]) 0.17307538323892796
np.exp(logprob_Ts[s][t]) 0.15588527506618216
np.exp(logprob_Ts[s][t]) 0.7214909148346109
np.exp(logprob_Ts[s][t]) 0.12262381009920703
np.exp(logprob_Ts[s][t]) 0.11377672570370621
np.exp(logprob_Ts[s][t]) 0.5265979989450715
np.exp(logprob_Ts[s][t]) 0.3596252753512221
np.exp(logprob_Ts[s][t]) 0.03546464335316357
np.exp(logprob_Ts[s][t]) 0.11209658260273596
np.exp(logprob_Ts[s][t]) 0.8524387740441005
np.exp(logprob_Ts[s][t]) 0.040951547978200095
np.exp(logprob_Ts[s][t]) 0.31141616045031006
np.exp(logprob_Ts[s][t]) 0.1104379491739671
np.exp(logprob_Ts[s][t]) 0.5371943423975228
np.exp(logprob_Ts[s][t]) 0.7382081941258597
np.exp(logprob_Ts[s][t]) 0.26179180587414036
np.exp(logprob_Ts[s][t]) 0.31504540233895034
np.exp(logprob_Ts[s][t]) 0.11172499230833147
np.exp(logprob_Ts[s][t]) 0.5434548017357619
np.exp(logprob_Ts[s][t]) 0.029774803616956416
np.exp(logprob_Ts[

[94418.58521193822,
 325121.41229417035,
 99585.98385676125,
 119638.46661803014,
 27891.055153880494,
 83803.37110737103,
 3591.414938186735,
 88606.30524181349,
 98514.79713156736,
 58828.60844628089]

In [1079]:
def assign_counts(logprob_Ts, reads, S):
    counts = [0] * 10
    
    for s in logprob_Ts:
        read_s = reads[s]
        for t in range(0, len(logprob_Ts[s])):
            if logprob_Ts[s][t] != 0:
                print('np.exp(logprob_Ts[s][t])',np.exp(logprob_Ts[s][t]))
                counts[t] += read_s * np.exp(logprob_Ts[s][t])
                
    return counts

In [1080]:
#Mpresley 5/12/23, attached is the maxmize step
def Maxmization2(counts):
    #postprobs is a dictionary
    # The 1st step is to convert the postprobs dictionary into a panda dataframe
    total = sum(counts)
    
    for i in range(len(counts)):
        counts[i] = counts[i]/total
    return counts